1. impot libraries
3. choose stocks and timeframe 
2. load data
3. transform to log returns
4. assign weights for portfolio return cretion
5. rolling predictions for parametric var and cvar with 252 days window and normal distribution
    1) 

In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm, t

In [2]:
tickers = ['AAPL', 'JNJ', 'XOM', 'JPM', 'WMT']

data = yf.download(tickers, period='2y', interval='1d')['Close']

data = data.dropna()

closing_prices = data[-501:]

print(closing_prices)

/var/folders/g3/d8cr9sbj1nq0g6fwxyhphc380000gn/T/ipykernel_20773/3731822354.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, period='2y', interval='1d')['Close']
[*********************100%***********************]  5 of 5 completed

Ticker            AAPL         JNJ         JPM        WMT         XOM
Date                                                                 
2023-07-27  191.304596  163.103378  148.849503  51.778225   98.402878
2023-07-28  193.888733  163.845245  149.698578  52.022221   97.226761
2023-07-31  194.502579  157.318848  150.700348  52.005951  100.101738
2023-08-01  193.670914  158.595963  149.956177  51.761967   99.523010
2023-08-02  190.670959  159.553772  148.257996  51.797749   98.281532
...                ...         ...         ...        ...         ...
2025-07-21  212.479996  164.360001  290.970001  95.669998  108.050003
2025-07-22  214.399994  167.929993  291.429993  95.849998  108.540001
2025-07-23  214.149994  169.100006  296.760010  95.680000  109.930000
2025-07-24  213.759995  169.559998  296.549988  96.599998  110.790001
2025-07-25  213.884995  168.089996  298.161804  97.464996  110.849998

[501 rows x 5 columns]


In [3]:
log_returns = np.log(closing_prices/closing_prices.shift(1)).dropna()

In [4]:
log_returns

Ticker,AAPL,JNJ,JPM,WMT,XOM
Date,,,,,
2023-07-28,0.013418,0.004538,0.005688,0.004701,-0.012024
2023-07-31,0.003161,-0.040648,0.006670,-0.000313,0.029141
2023-08-01,-0.004285,0.008085,-0.004950,-0.004703,-0.005798
2023-08-02,-0.015611,0.006021,-0.011389,0.000691,-0.012553
2023-08-03,-0.007349,0.004287,0.006095,0.000251,0.017231
...,...,...,...,...,...
2025-07-21,0.006137,0.004024,-0.001030,0.006502,0.002595
2025-07-22,0.008996,0.021488,0.001580,0.001880,0.004525
2025-07-23,-0.001167,0.006943,0.018124,-0.001775,0.012725


In [18]:
window = 250

alpha_95 = 0.95
alpha_99 = 0.99

z_score_95 = norm.ppf(alpha_95)
z_score_99 = norm.ppf(alpha_99)

weights = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

In [ ]:
par_rolling_var_95 = []
par_rolling_var_99 = []

for t in range(window, len(log_returns)):
    window_log_returns = log_returns.iloc[t-window:t]
    cov_matrix = window_log_returns.cov().values
    portfolio_std = np.sqrt(weights.T @ cov_matrix @ weights)
    var_95 = z_score_95 * portfolio_std
    var_99 = z_score_99 * portfolio_std
    par_rolling_var_95.append(var_95) # assume mean = 0 due to negligible effect in daily log returns
    par_rolling_var_99.append(var_99)

In [22]:
var_95_series = pd.Series(par_rolling_var_95, index=log_returns.index[window:])
var_99_series = pd.Series(par_rolling_var_99, index=log_returns.index[window:])

In [23]:
realized_returns = log_returns @ weights
realized_returns = realized_returns[window:]

In [25]:
par_breaches_95 = realized_returns < -var_95_series
par_breaches_99 = realized_returns < -var_99_series

par_breach_rate_95 = par_breaches_95.sum()/len(par_breaches_95)
par_breach_rate_99 = par_breaches_99.sum()/len(par_breaches_99)

print(f'breach_rate_95: {par_breach_rate_95:.2%}')
print(f'breach_rate_99: {par_breach_rate_99:.2%}')

breach_rate_95: 6.40%
breach_rate_99: 3.20%


In [28]:
from scipy.stats import chi2

T = len(var_95_series)
x = len(par_breaches_95)
p = alpha_95  # nominal exception rate, e.g. 0.05

hat_p = x / T
L0 = ((1-p)**(T-x)) * (p**x)
L1 = ((1-hat_p)**(T-x)) * (hat_p**x)
LR_pof = -2 * np.log(L0 / L1)
p_value = 1 - chi2.cdf(LR_pof, df=1)

print(p_value)

4.1000723660644667e-07


In [31]:
T = len(var_99_series)
x = len(par_breaches_99)
p = alpha_99  # nominal exception rate, e.g. 0.05

hat_p = x / T
L0 = ((1-p)**(T-x)) * (p**x)
L1 = ((1-hat_p)**(T-x)) * (hat_p**x)
LR_pof = -2 * np.log(L0 / L1)
p_value = 1 - chi2.cdf(LR_pof, df=1)
print(p_value)

0.024981503053449705
